## Start dataset

In [ ]:
import pandas as pd

In [ ]:
d = pd.read_csv("/home/yagor/Рабочий стол/mipt/lab3/notebook/nutrition_table.csv")
d_col = list(d.columns)
d_col

['Unnamed: 0',
 'fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'proteins_100g',
 'salt_100g',
 'energy_100g',
 'reconstructed_energy',
 'g_sum',
 'exceeded',
 'product']

In [ ]:
d

,Unnamed: 0,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,energy_100g,reconstructed_energy,g_sum,exceeded,product
0,1,28.57,64.29,14.29,3.57,0.00000,2243.0,2267.85,96.43,0,Banana Chips Sweetened (Whole)
1,2,17.86,60.71,17.86,17.86,0.63500,1941.0,2032.23,96.43,0,Peanuts
2,3,57.14,17.86,3.57,17.86,1.22428,2540.0,2835.70,92.86,0,Organic Salted Nut Mix
3,7,18.75,57.81,15.62,14.06,0.13970,1833.0,1953.04,90.62,0,Organic Muesli
4,12,36.67,36.67,3.33,16.67,1.60782,2230.0,2336.91,90.01,0,Zen Party Mix
...,...,...,...,...,...,...,...,...,...,...,...
45023,49995,0.00,17.92,7.08,0.42,0.03810,297.0,311.78,18.34,0,"100% Juice, Prune Juice"
45024,49996,0.00,14.58,14.58,0.00,0.03810,243.0,247.86,14.58,0,100% Juice Blend
45025,49997,0.00,13.33,13.33,0.00,0.03810,226.0,226.61,13.33,0,Cranberry Apple Flavored Juice Blended With On...
45026,49998,0.00,16.25,16.25,0.00,0.03810,259.0,276.25,16.25,0,"Cranberry Grape, 100% Juice Blend"


## Load data from HBase

In [292]:
import happybase

connection = happybase.Connection('hbase-docker', 9090)
table = connection.table('nutrition')

In [293]:
all_data = {}
for key, data in table.scan():
    all_data[key] = data 

In [294]:
all_data[b'1']

{b'nutrition_data:carbohydrates_100g': b'64.29',
 b'nutrition_data:energy_100g': b'2243.0',
 b'nutrition_data:exceeded': b'0',
 b'nutrition_data:fat_100g': b'28.57',
 b'nutrition_data:g_sum': b'96.43',
 b'nutrition_data:product': b'Banana Chips Sweetened (Whole)',
 b'nutrition_data:proteins_100g': b'3.57',
 b'nutrition_data:reconstructed_energy': b'2267.85',
 b'nutrition_data:salt_100g': b'0.0',
 b'nutrition_data:sugars_100g': b'14.29'}

In [295]:
all_data[b'1'].keys()

dict_keys([b'nutrition_data:carbohydrates_100g', b'nutrition_data:energy_100g', b'nutrition_data:exceeded', b'nutrition_data:fat_100g', b'nutrition_data:g_sum', b'nutrition_data:product', b'nutrition_data:proteins_100g', b'nutrition_data:reconstructed_energy', b'nutrition_data:salt_100g', b'nutrition_data:sugars_100g'])

In [296]:
decode_all_data = []
funcs = {'nutrition_data:exceeded': int, 'nutrition_data:product': str}
for key in all_data.keys():
    decode_row_data = {}
    for second_key, val in all_data[key].items():
        k = second_key.decode("utf-8")
        v = val.decode("utf-8")
        if k in funcs:
            v = funcs[k](v)
        else:
            v = float(v)
        decode_row_data[k.split(':')[-1]] = v
    decode_all_data.append(decode_row_data)



In [297]:
decode_all_data[0]

{'carbohydrates_100g': 64.29,
 'energy_100g': 2243.0,
 'exceeded': 0,
 'fat_100g': 28.57,
 'g_sum': 96.43,
 'product': 'Banana Chips Sweetened (Whole)',
 'proteins_100g': 3.57,
 'reconstructed_energy': 2267.85,
 'salt_100g': 0.0,
 'sugars_100g': 14.29}

In [298]:
import csv
import pandas as pd

keys = decode_all_data[0].keys()

with open('data_from_hbase.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(decode_all_data)

In [299]:
data_from_db = pd.read_csv('/home/yagor/Рабочий стол/mipt/lab3/notebook/data_from_hbase.csv')
data_from_db.head()

,carbohydrates_100g,energy_100g,exceeded,fat_100g,g_sum,product,proteins_100g,reconstructed_energy,salt_100g,sugars_100g
0,64.29,2243.0,0,28.57,96.43,Banana Chips Sweetened (Whole),3.57,2267.85,0.00000,14.29
1,80.00,1255.0,0,0.00,82.50,California Seedless Raisins,2.50,1402.50,0.00000,60.00
2,73.33,1536.0,0,8.33,84.99,Tropical Fruit Mix,3.33,1628.09,0.12700,56.67
3,0.00,33.0,0,0.41,1.63,Beef Broth,1.22,36.73,0.88138,0.00
4,21.43,1197.0,0,3.57,67.86,Beef Jerky,42.86,1232.16,4.35356,17.86


In [300]:
data_from_db.to_csv('data_from_hbase_with_postprocess.csv')

In [301]:
data_from_db_new = pd.read_csv('/home/yagor/Рабочий стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv')
data_from_db_new.head()

,Unnamed: 0,carbohydrates_100g,energy_100g,exceeded,fat_100g,g_sum,product,proteins_100g,reconstructed_energy,salt_100g,sugars_100g
0,0,64.29,2243.0,0,28.57,96.43,Banana Chips Sweetened (Whole),3.57,2267.85,0.00000,14.29
1,1,80.00,1255.0,0,0.00,82.50,California Seedless Raisins,2.50,1402.50,0.00000,60.00
2,2,73.33,1536.0,0,8.33,84.99,Tropical Fruit Mix,3.33,1628.09,0.12700,56.67
3,3,0.00,33.0,0,0.41,1.63,Beef Broth,1.22,36.73,0.88138,0.00
4,4,21.43,1197.0,0,3.57,67.86,Beef Jerky,42.86,1232.16,4.35356,17.86


## Pyspark clastering

In [90]:
MASTER = "local"
NUM_PROCESSORS = "8"
NUM_EXECUTORS = "4"
NUM_PARTITIONS = 10

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import pyspark.sql.functions as psf
import json

In [91]:
conf = SparkConf()

conf.set("spark.app.name", "one_part_data")
conf.set("spark.master", MASTER)
conf.set("spark.executor.cores", NUM_PROCESSORS)
conf.set("spark.executor.instances", NUM_EXECUTORS)
conf.set("spark.executor.memory", "6g")
conf.set("spark.locality.wait", "0")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.kryoserializer.buffer.max", "2000")
conf.set("spark.executor.heartbeatInterval", "6000s")
conf.set("spark.network.timeout", "10000000s")
conf.set("spark.shuffle.spill", "true")
conf.set("spark.driver.memory", "15g")
conf.set("spark.driver.maxResultSize", "15g")

In [92]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .config(conf=conf) \
      .master("local[*]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()
spark

In [93]:
dataset = spark.read.csv("/home/yagor/Рабочий стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv",header=True,inferSchema=True)

In [94]:
dataset.show()

22/12/29 23:35:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
 Schema: _c0, carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
Expected: _c0 but found: 
CSV file: file:///home/yagor/Рабочий%20стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+---------+-----------+
|_c0|carbohydrates_100g|energy_100g|exceeded|fat_100g|g_sum|             product|proteins_100g|reconstructed_energy|salt_100g|sugars_100g|
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+---------+-----------+
|  0|             64.29|     2243.0|       0|   28.57|96.43|Banana Chips Swee...|         

In [95]:
dataset.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- carbohydrates_100g: double (nullable = true)
 |-- energy_100g: double (nullable = true)
 |-- exceeded: integer (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- g_sum: double (nullable = true)
 |-- product: string (nullable = true)
 |-- proteins_100g: double (nullable = true)
 |-- reconstructed_energy: double (nullable = true)
 |-- salt_100g: double (nullable = true)
 |-- sugars_100g: double (nullable = true)



In [96]:
dataset.columns

['_c0',
 'carbohydrates_100g',
 'energy_100g',
 'exceeded',
 'fat_100g',
 'g_sum',
 'product',
 'proteins_100g',
 'reconstructed_energy',
 'salt_100g',
 'sugars_100g']

In [97]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [98]:
feat_cols = [ #'_c0',
 'fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'proteins_100g',
 'salt_100g',
 'energy_100g',
 'reconstructed_energy',
 'g_sum',
 'exceeded',
 #'product'
 ]

In [99]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [100]:
final_data = vec_assembler.transform(dataset)

In [101]:
final_data.show()

22/12/29 23:35:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
 Schema: _c0, carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
Expected: _c0 but found: 
CSV file: file:///home/yagor/Рабочий%20стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+---------+-----------+--------------------+
|_c0|carbohydrates_100g|energy_100g|exceeded|fat_100g|g_sum|             product|proteins_100g|reconstructed_energy|salt_100g|sugars_100g|            features|
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+---------+-----------+--------------------+
|  0|             64.29|   

In [102]:
from pyspark.ml.feature import StandardScaler

In [103]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [104]:
final_data

DataFrame[_c0: int, carbohydrates_100g: double, energy_100g: double, exceeded: int, fat_100g: double, g_sum: double, product: string, proteins_100g: double, reconstructed_energy: double, salt_100g: double, sugars_100g: double, features: vector]

In [105]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [106]:
# Normalize each feature to have unit standard deviation.
cluster_final_data = scalerModel.transform(final_data)

## train model

In [107]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [120]:
for k in range(2,15):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    predictions = model.transform(cluster_final_data)
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    print("With K={}".format(k))
    print("Silhouette with squared euclidean distance = " + str(silhouette))
    print('--'*30)

With K=2
Silhouette with squared euclidean distance = 0.759665199433813
------------------------------------------------------------
With K=3
Silhouette with squared euclidean distance = 0.5478606195285595
------------------------------------------------------------
With K=4
Silhouette with squared euclidean distance = 0.5443140985003554
------------------------------------------------------------
With K=5
Silhouette with squared euclidean distance = 0.522830972258517
------------------------------------------------------------
With K=6
Silhouette with squared euclidean distance = 0.3218162020176603
------------------------------------------------------------
With K=7
Silhouette with squared euclidean distance = 0.2837808134803726
------------------------------------------------------------
With K=8
Silhouette with squared euclidean distance = 0.2719213425157517
------------------------------------------------------------
With K=9
Silhouette with squared euclidean distance = 0.33359760

In [383]:
k = 8
kmeans = KMeans(featuresCol='scaledFeatures',k=k)

In [384]:
model = kmeans.fit(cluster_final_data)

In [385]:
predictions = model.transform(cluster_final_data)

In [386]:
predictions.show()

22/12/30 01:36:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
 Schema: _c0, carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, product, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
Expected: _c0 but found: 
CSV file: file:///home/yagor/Рабочий%20стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+---------+-----------+--------------------+--------------------+----------+
|_c0|carbohydrates_100g|energy_100g|exceeded|fat_100g|g_sum|             product|proteins_100g|reconstructed_energy|salt_100g|sugars_100g|            features|      scaledFeatures|prediction|
+---+------------------+-----------+--------+--------+-----+--------------------+-------------+--------------------+-------

In [387]:
evaluator = ClusteringEvaluator()

In [388]:
silhouette = evaluator.evaluate(predictions)

In [389]:
print(f"With k={k} Silhouette with squared euclidean distance = " + str(silhouette))

With k=8 Silhouette with squared euclidean distance = 0.2719213425157517


In [390]:
centers=model.clusterCenters()
print("Cluster Centers:")
for center in centers:
    print(center)

Cluster Centers:
[ 1.96583064  2.21301295  1.56877413  1.41905729  0.06721025  2.94866513
  3.12098855  3.17007092 11.14337555]
[0.08476997 0.39927264 0.32057744 0.24624109 0.09684325 0.37789641
 0.3565178  0.44537065 0.        ]
[0.9277834  0.46578902 0.23855921 1.52108774 0.1101182  1.28026966
 1.26033541 1.20040032 0.        ]
[0.11400264 2.7799988  3.03181841 0.20737088 0.03800656 1.87571684
 1.85788713 2.51609382 0.        ]
[ 1.002298    0.8864475   0.39101108  0.88978501 85.41558104  1.45696644
  1.4693043   1.4466644   0.        ]
[0.28764143 2.23122392 0.34957771 1.21711714 0.15304523 1.82944102
 1.82333719 2.36463893 0.        ]
[1.67052667 1.97662047 1.37275275 0.71507318 0.07159686 2.63495316
 2.62646794 2.65813688 0.        ]
[2.884237   0.40454504 0.13832303 2.50930314 0.11063761 2.82465663
 2.87952476 2.28732615 0.        ]


In [391]:
model.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|11531|
|         6| 3765|
|         3| 2940|
|         5| 4130|
|         4|    3|
|         7| 2727|
|         2| 4958|
|         0|  246|
+----------+-----+



In [381]:
cols = ["_c0", "prediction"]
result = predictions.select(*cols)

In [382]:
result.show()

22/12/30 01:26:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
 Schema: _c0, carbohydrates_100g, energy_100g, exceeded, fat_100g, g_sum, proteins_100g, reconstructed_energy, salt_100g, sugars_100g
Expected: _c0 but found: 
CSV file: file:///home/yagor/Рабочий%20стол/mipt/lab3/notebook/data_from_hbase_with_postprocess.csv
+---+----------+
|_c0|prediction|
+---+----------+
|  0|         0|
|  1|         6|
|  2|         6|
|  3|         1|
|  4|         4|
|  5|         0|
|  6|         6|
|  7|         1|
|  8|         6|
|  9|         6|
| 10|         1|
| 11|         6|
| 12|         6|
| 13|         6|
| 14|         6|
| 15|         6|
| 16|         6|
| 17|         5|
| 18|         0|
| 19|         1|
+---+----------+
only showing top 20 rows



In [ ]:
result.write.format("com.databricks.spark.csv") \
  .option("header", "true") \
  .save("output_result.csv")

## Load result from hbase

In [190]:
import happybase

connection = happybase.Connection('hbase-docker', 9090)
table = connection.table('result')

In [191]:
all_data = {}
for key, data in table.scan():
    all_data[key] = data 

In [203]:
len(all_data)

45029

In [193]:
all_data[b'1']

{b'result_data:index': b'2'}

In [189]:
all_data[b'1'].keys()

dict_keys([b'res_data:index'])

In [212]:
decode_all_data = []
funcs = {'result_data:prediction': int}
for key in all_data.keys():
    decode_row_data = {}
    for second_key, val in all_data[key].items():
        k = second_key.decode("utf-8")
        v = val.decode("utf-8")
        try:
            cluster = int(v)
            decode_row_data['prediction'] = cluster
        except:
            print(f'key:{k}, value:{v} not int')
    try:
        decode_row_data['index'] = int(key.decode("utf-8"))
    except:
        print(f'key:{key.decode("utf-8")} not int')
    decode_all_data.append(decode_row_data)



key:result_data:index, value:prediction not int
key:_c0 not int


In [213]:
decode_all_data[1]

{'prediction': 3, 'index': 10003}

In [242]:
import csv
import pandas as pd

keys = decode_all_data[0].keys()

with open('data_from_hbase_result.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(decode_all_data)

In [257]:
data_from_db = pd.read_csv('/home/yagor/Рабочий стол/mipt/lab3/notebook/data_from_hbase_result.csv').astype("Int64")
data_from_db.head()

,prediction,index
0,2,1
1,3,10003
2,0,10004
3,0,10005
4,0,10006


In [ ]:
data_from_db.set_index('index')

In [286]:
df = data_from_db[data_from_db['index'].notna()].set_index('index').sort_index()
df

,prediction
index,
1,2
2,2
3,5
7,2
12,5
...,...
49995,0
49996,0
49997,0


In [287]:
df.to_csv('data_result_from_hbase_with_postprocess.csv')

In [290]:
data_from_db_new_res = pd.read_csv('/home/yagor/Рабочий стол/mipt/lab3/notebook/data_result_from_hbase_with_postprocess.csv', index_col=['index'])
data_from_db_new_res.head()

,prediction
index,
1,2
2,2
3,5
7,2
12,5


In [291]:
data_from_db_new_res.count()

prediction    45028
dtype: int64

## Load in and out HBase data (experiments)

In [38]:
import pandas as pd

In [303]:
d = pd.read_csv("/home/yagor/Рабочий стол/mipt/lab3/notebook/nutrition_table.csv")
d_col = list(d.columns)
d_col

['Unnamed: 0',
 'fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'proteins_100g',
 'salt_100g',
 'energy_100g',
 'reconstructed_energy',
 'g_sum',
 'exceeded',
 'product']

In [305]:
d

,Unnamed: 0,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,energy_100g,reconstructed_energy,g_sum,exceeded,product
0,1,28.57,64.29,14.29,3.57,0.00000,2243.0,2267.85,96.43,0,Banana Chips Sweetened (Whole)
1,2,17.86,60.71,17.86,17.86,0.63500,1941.0,2032.23,96.43,0,Peanuts
2,3,57.14,17.86,3.57,17.86,1.22428,2540.0,2835.70,92.86,0,Organic Salted Nut Mix
3,7,18.75,57.81,15.62,14.06,0.13970,1833.0,1953.04,90.62,0,Organic Muesli
4,12,36.67,36.67,3.33,16.67,1.60782,2230.0,2336.91,90.01,0,Zen Party Mix
...,...,...,...,...,...,...,...,...,...,...,...
45023,49995,0.00,17.92,7.08,0.42,0.03810,297.0,311.78,18.34,0,"100% Juice, Prune Juice"
45024,49996,0.00,14.58,14.58,0.00,0.03810,243.0,247.86,14.58,0,100% Juice Blend
45025,49997,0.00,13.33,13.33,0.00,0.03810,226.0,226.61,13.33,0,Cranberry Apple Flavored Juice Blended With On...
45026,49998,0.00,16.25,16.25,0.00,0.03810,259.0,276.25,16.25,0,"Cranberry Grape, 100% Juice Blend"


In [45]:
host = 'hbase-docker'#"172.17.0.2"#, 9090
port = 9090
def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    conn = happybase.Connection(host = host, port=port, autoconnect=True,
        table_prefix = namespace,
        table_prefix_separator = ":")
    conn.open()
    table = conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return conn, batch

In [54]:
conn, batch = connect_to_hbase()

In [55]:
conn, batch

(<happybase.connection.Connection at 0x7f5a0488b9a0>,
 <happybase.batch.Batch at 0x7f5a04888610>)

In [56]:
conn.table

<bound method Connection.table of <happybase.connection.Connection object at 0x7f5a0488b9a0>>

In [57]:
conn.is_table_enabled(table_name)

IOError: IOError(message=b'org.apache.hadoop.hbase.TableNotFoundException: sample_data:nutrition\n\tat org.apache.hadoop.hbase.client.HBaseAdmin$42.rpcCall(HBaseAdmin.java:2048)\n\tat org.apache.hadoop.hbase.client.HBaseAdmin$42.rpcCall(HBaseAdmin.java:2044)\n\tat org.apache.hadoop.hbase.client.RpcRetryingCallable.call(RpcRetryingCallable.java:58)\n\tat org.apache.hadoop.hbase.client.RpcRetryingCallerImpl.callWithRetries(RpcRetryingCallerImpl.java:107)\n\tat org.apache.hadoop.hbase.client.HBaseAdmin.executeCallable(HBaseAdmin.java:3084)\n\tat org.apache.hadoop.hbase.client.HBaseAdmin.executeCallable(HBaseAdmin.java:3076)\n\tat org.apache.hadoop.hbase.client.HBaseAdmin.checkTableExists(HBaseAdmin.java:2044)\n\tat org.apache.hadoop.hbase.client.HBaseAdmin.isTableEnabled(HBaseAdmin.java:971)\n\tat org.apache.hadoop.hbase.thrift.ThriftServerRunner$HBaseHandler.isTableEnabled(ThriftServerRunner.java:872)\n\tat sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat sun.reflect.Nat

In [43]:
conn.create_table(table_name, { 'data': dict() } )

IOError: IOError(message=b'org.apache.hadoop.hbase.NamespaceNotFoundException: org.apache.hadoop.hbase.NamespaceNotFoundException: sample_data\n\tat org.apache.hadoop.hbase.master.ClusterSchemaServiceImpl.getNamespace(ClusterSchemaServiceImpl.java:119)\n\tat org.apache.hadoop.hbase.master.HMaster.createTable(HMaster.java:1975)\n\tat org.apache.hadoop.hbase.master.MasterRpcServices.createTable(MasterRpcServices.java:630)\n\tat org.apache.hadoop.hbase.shaded.protobuf.generated.MasterProtos$MasterService$2.callBlockingMethod(MasterProtos.java)\n\tat org.apache.hadoop.hbase.ipc.RpcServer.call(RpcServer.java:413)\n\tat org.apache.hadoop.hbase.ipc.CallRunner.run(CallRunner.java:130)\n\tat org.apache.hadoop.hbase.ipc.RpcExecutor$Handler.run(RpcExecutor.java:324)\n\tat org.apache.hadoop.hbase.ipc.RpcExecutor$Handler.run(RpcExecutor.java:304)\n\n\tat sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)\n\tat sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructo

In [35]:
"""
Insert data into HBase with a Python script.
To create the table, first use the hbase shell. We are going to create a
namespace called "sample_data". The table for this script is called "rfic",
as we will be inserting Request for Information Cases from the City of
Indianapolis.
Our table will have only one column family named "data", and we are accepting
all table defaults.
Original data source
https://data.indy.gov/dataset/Request-for-Information-Cases/ts4b-8qa9
% hbase shell
hbase> create_namespace "sample_data"
hbase> create "sample_data:rfic", "data"
"""

import csv
import happybase
import time

batch_size = 1000
host = 'hbase-docker'#"172.17.0.2"#, 9090
port = 9090
file_path = "/home/yagor/Рабочий стол/mipt/lab3/notebook/nutrition_table.csv"
namespace = "sample_data"
row_count = 0
start_time = time.time()
table_name = "nutrition"


def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    
    conn = happybase.Connection(host = host, port=port,
        table_prefix = namespace,
        table_prefix_separator = ":")
    conn.open()
    table = conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return conn, batch


def insert_row(batch, row):
    """ Insert a row into HBase.
 Write the row to the batch. When the batch size is reached, rows will be
 sent to the database.
 Rows have the following schema:
 [ id, keyword, subcategory, type, township, city, zip, council_district,
 opened, closed, status, origin, location ]
    """
    # print(type(row[0]))
    batch.put(row[0], { "data:fat_100g": row[1], "data:carbohydrates_100g": row[2], "data:sugars_100g": row[3],
        "data:proteins_100g": row[4], "data:salt_100g": row [5], "data:energy_100g": row[6],
        "data:reconstructed_energy": row[7], "data:g_sum": row[8], "data:exceeded": row[9],
        "data:product": row[10]})


def read_csv():
    csvfile = open(file_path, "r")
    csvreader = csv.reader(csvfile)
    return csvreader, csvfile


# After everything has been defined, run the script.
conn, batch = connect_to_hbase()
print(f"Connect to HBase. table name: {table_name}, batch size: {batch_size}")
csvreader, csvfile = read_csv()
print(f"Connected to file. name: {file_path}")

try:
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    for row in csvreader:
        row_count += 1
        if row_count == 1:
            pass
        else:
            insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
    csvfile.close()
    conn.close()

duration = time.time() - start_time
print(f"Done. row count: {row_count}, duration: {duration} s".format(row_count, duration))

Connect to HBase. table name: nutrition, batch size: 1000
Connected to file. name: /home/yagor/Рабочий стол/mipt/lab3/notebook/nutrition_table.csv


IOError: IOError(message=b'org.apache.hadoop.hbase.client.RetriesExhaustedWithDetailsException: Failed 100 actions: sample_data:nutrition: 100 times, servers with issues: null\n\tat org.apache.hadoop.hbase.client.BatchErrors.makeException(BatchErrors.java:54)\n\tat org.apache.hadoop.hbase.client.AsyncRequestFutureImpl.getErrors(AsyncRequestFutureImpl.java:1227)\n\tat org.apache.hadoop.hbase.client.HTable.batch(HTable.java:455)\n\tat org.apache.hadoop.hbase.client.HTable.put(HTable.java:553)\n\tat org.apache.hadoop.hbase.thrift.ThriftServerRunner$HBaseHandler.mutateRowsTs(ThriftServerRunner.java:1471)\n\tat org.apache.hadoop.hbase.thrift.ThriftServerRunner$HBaseHandler.mutateRows(ThriftServerRunner.java:1407)\n\tat sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.lang.reflect.Method.invoke(Me

In [58]:

from thrift import Thrift
from thrift.transport import TSocket, TTransport
from thrift.protocol import TBinaryProtocol
 
 # connect 
tsocket = TSocket.TSocket(host = host, port=port)
tsocket.setTimeout(5000)
ttransport = TTransport.TBufferedTransport(tsocket)
protocol = TBinaryProtocol.TBinaryProtocol(ttransport)
 
from hbase import Hbase
 
 #  
client = Hbase.Client(protocol)
tsocket.open()
 
 #    
tableName = bytes("namespace:table", encoding='utf8')
rowkey = bytes("rowkey", encoding='utf8')
 
 # Get all table names
tableNames = client.getTableNames()
print('tableNames:',tableNames)
 
 #      
columnDescriptors = client.getColumnDescriptors(tableName)
print("columnName:",columnDescriptors)
 
 #    
row = client.getRow(tableName,rowkey,attributes={})


ModuleNotFoundError: No module named 'thrift'

In [27]:
import happybase

# Подключиться к базе данных
connection = happybase.Connection('hbase-docker', 9090)

# Запросить все таблицы
table_name_list = connection.tables()

# Построить таблицу
families = {
	'user_info': dict(),
	'history': dict()
}

connection.create_table('table_name', families)

In [34]:
import happybase

connection = happybase.Connection('hbase-docker', 9090)
table = connection.table('nutrition')






In [35]:
all_data = {}
for key, data in table.scan():
    all_data[key] = data 

In [50]:
all_data[b'1']

{b'nutrition_data:carbohydrates_100g': b'64.29',
 b'nutrition_data:energy_100g': b'2243.0',
 b'nutrition_data:exceeded': b'0',
 b'nutrition_data:fat_100g': b'28.57',
 b'nutrition_data:g_sum': b'96.43',
 b'nutrition_data:product': b'Banana Chips Sweetened (Whole)',
 b'nutrition_data:proteins_100g': b'3.57',
 b'nutrition_data:reconstructed_energy': b'2267.85',
 b'nutrition_data:salt_100g': b'0.0',
 b'nutrition_data:sugars_100g': b'14.29'}

In [51]:
all_data[b'1'].keys()

dict_keys([b'nutrition_data:carbohydrates_100g', b'nutrition_data:energy_100g', b'nutrition_data:exceeded', b'nutrition_data:fat_100g', b'nutrition_data:g_sum', b'nutrition_data:product', b'nutrition_data:proteins_100g', b'nutrition_data:reconstructed_energy', b'nutrition_data:salt_100g', b'nutrition_data:sugars_100g'])

In [56]:
decode_all_data = []
funcs = {'nutrition_data:exceeded': int, 'nutrition_data:product': str}
for key in all_data.keys():
    decode_row_data = {}
    for second_key, val in all_data[key].items():
        k = second_key.decode("utf-8")
        v = val.decode("utf-8")
        if k in funcs:
            v = funcs[k](v)
        else:
            v = float(v)
        decode_row_data[k.split(':')[-1]] = v
    decode_row_data['index'] = int(key.decode("utf-8"))
    decode_all_data.append(decode_row_data)



In [57]:
decode_all_data[0]

{'carbohydrates_100g': 64.29,
 'energy_100g': 2243.0,
 'exceeded': 0,
 'fat_100g': 28.57,
 'g_sum': 96.43,
 'product': 'Banana Chips Sweetened (Whole)',
 'proteins_100g': 3.57,
 'reconstructed_energy': 2267.85,
 'salt_100g': 0.0,
 'sugars_100g': 14.29,
 'index': 1}

In [58]:
import csv

keys = decode_all_data[0].keys()

with open('data_from_hbase.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(decode_all_data)

In [65]:
data_from_db = pd.read_csv('/home/yagor/Рабочий стол/mipt/lab3/notebook/data_from_hbase.csv')
data_from_db.head()

,carbohydrates_100g,energy_100g,exceeded,fat_100g,g_sum,product,proteins_100g,reconstructed_energy,salt_100g,sugars_100g,index
0,64.29,2243.0,0,28.57,96.43,Banana Chips Sweetened (Whole),3.57,2267.85,0.00000,14.29,1
1,80.00,1255.0,0,0.00,82.50,California Seedless Raisins,2.50,1402.50,0.00000,60.00,10003
2,73.33,1536.0,0,8.33,84.99,Tropical Fruit Mix,3.33,1628.09,0.12700,56.67,10007
3,0.00,33.0,0,0.41,1.63,Beef Broth,1.22,36.73,0.88138,0.00,10008
4,21.43,1197.0,0,3.57,67.86,Beef Jerky,42.86,1232.16,4.35356,17.86,10009


In [66]:
data_from_db.to_csv('data_from_hbase_with_postprocess.csv')

In [55]:
decode_all_data.keys()

dict_keys(['nutrition_data:carbohydrates_100g', 'nutrition_data:energy_100g', 'nutrition_data:exceeded', 'nutrition_data:fat_100g', 'nutrition_data:g_sum', 'nutrition_data:product', 'nutrition_data:proteins_100g', 'nutrition_data:reconstructed_energy', 'nutrition_data:salt_100g', 'nutrition_data:sugars_100g'])

In [48]:
all_data[b'1'][b'nutrition_data:carbohydrates_100g'].decode("utf-8")

'64.29'

In [ ]:
for key, data in table.scan():
    print(key, data)  # prints 'value1' and 'value2'

In [25]:
connection.disable_table('table_name')

In [26]:
connection.delete_table('table_name', disable=True)

In [28]:
table = connection.table('table_name')

# Получить информацию о семействе столбцов
info_dict = table.families()


